In [1]:
import os
import pandas as pd
import pickle
from time import time
from tqdm import tqdm

pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_columns", 500)

# custom imports
from data_handling import load_data, collapse_dfs
from text_preprocessing import df_processing

In [2]:
read_in = False
path = '/home/franzi/Git-DSR/Twitter_sentimentanalysis/Data/annotation_data.pickle'
parties = ['AfD', 'SPD', 'CDU', 'CSU', 'FDP', 'LINKE', 'Gruene', 'Grüne']
dates = ['08-{:02d}'.format(i) for i in range(2,3)]


if read_in:
    parties = parties
    dates = dates
    aug_data = {}
    print('Loading: {}'.format(dates))
    for party in parties:
        load_list = ['../Data/{}/{}_2018-{}.json'.format(party, party, day) for day in dates]
        aug_data[party] = load_data(load_list)
        print('{} loaded'.format(party))
    with open(path, 'wb') as fid:
        pickle.dump(aug_data, fid)
    print("Done")
else:
    print('loading data from pickle')
    with open(path, 'rb') as fid:
        aug_data = pickle.load(fid)
        print('data loaded')

loading data from pickle
data loaded


In [3]:
collapse = False
path_collapse = '/home/franzi/Git-DSR/Twitter_sentimentanalysis/Data/all_data_annotation.pickle' # path where pickle file is saved

if collapse:
    all_data = collapse_dfs(aug_data)
    df_processing(all_data)
    all_data = all_data.drop_duplicates('clean_text')
    all_data['user_id'] = all_data.loc[:, 'user'].map(lambda x: x['id_str'])
    all_data.drop('user', inplace=True, axis=1)
    print('dumping data to pickle')
    with open(path_collapse, 'wb') as fid:
        pickle.dump(all_data, fid)
    print('data dumped')
else: 
    print('loading data from pickle')
    with open(path_collapse, 'rb') as fid:
        all_data = pickle.load(fid)
        print('data loaded')

loading data from pickle
data loaded


In [6]:
def annotate(dfin, no_annotations=10, old_annotations=pd.DataFrame()):
    old_text = []
    
    if not old_annotations.empty:
        old_text = old_annotations.iloc[:,[1,2]].values.tolist()
#         print(type(old_text))
#         print(old_text[1])
        results = list(old_text)
    else:
        results = []
        
    for i, text in enumerate(dfin.iloc[:no_annotations, 4]):
        if text in [e[0] for e in old_text]:
            if(i%10 == 0):
                  print (i)
            continue
        else:
            print(i)
            a = input(text)
            if a == "a":
                out = "pro AfD"
            elif a == "s":
                out = "contra AfD"
            elif a == "d":
                out = "contra Alles"
            elif a == "f":
                out = "unknown"
            elif a == "e":
                out = "not German"
            else:
                print("result not found")
            results.append([text, out])
    return results

In [11]:
annotations = annotate(all_data, no_annotations=100, old_annotations=old_annotations)

0
10
20
30
40
50
AfD  BW   mit  Dr   Marc  Jongen   Thomas  Seitz   MdB   Marc  Bernhard  MdB  und  Ma     einLink  via  f
51
   Sein  Anwalt  erklaerte  nun  gegenueber  dem  ZDF   dass  der  Islamist   SamiA  nur  noch  auf  die  Ausstellung  seines  neuen  Reisepass  f
52
   WAS   ZUM  FICK  IST  DAS  FUeR  EINE  FRAGE    Diese  Frage  kommt  auf   weil  die  AfD      faelschlicherweise      behauptet  hat  in  eine  s
53
   Herr   Erdogan  hat  keinen  Staatsbesuch  in  Deutschland  zu   planen   wie  ein  Sultan   der  eine  osmanische  Provinz  visitiert   d
54
Der  Landesvorsitzende  der  niedersaechsischen   AfD  Jugendorganisation  Junge  Alternative   Lars  Steinke   hat  den  Hitler  Attentaeter  Claus  Schenk  Graf  von  Stauffenberg  in  einem  Facebook  Post  als   Verraeter   und   Feigling   bezeichnet    fckafd   fcknzs  einLink  via  f
55
   Wenn  wir  die  EZB  nicht  zwingen  koennen   die  Regeln  einzuhalten   dann  muessen   wir  die  Notbremse  ziehen   amp   ra

In [12]:
annotations = pd.DataFrame(annotations)

In [13]:
annotations.to_csv("../Data/annotation.csv")

In [14]:
old_annotations = pd.read_csv("../Data/annotation.csv")

In [4]:
# old_annotations.to_csv("../Data/.csv")